# 第一版存取

In [1]:
import pymongo
from pymongo import MongoClient
import datetime
import json
import time
from functools import reduce

#連Mongodb
client = MongoClient('172.20.26.39', 27017)
#連DB名
db=client['rawData']
#秀DB內的collection list
db.collection_names()

['wowprimediendata', 'wowprimeipeenAddCommentURL', 'wowprimeipeen']

In [ ]:
#選擇我們想要的collection
collection = db.wowprimeipeen
#list(dict)的去重公式
f = lambda x,y:x if y in x else x + [y]

#不斷重複動作
while True:
    #讀取資料
    try:
        with open("../data/diendata.json","r") as d:
            diendata=json.load(d)
            print("讀取完畢")
            #做去重動作
            diendata = reduce(f, [[], ] + diendata)
            print("去重完畢")
            with open("../data/mongodiendata.json","w") as d:
                json.dump(diendata,d)
            print("另存完畢")
            #比數變多，則做儲存動作
            if len(diendata)>len(list(collection.find({}))):
                collection.drop()
                collection.insert_many(diendata)
                print("存資料庫完畢")
                print("目前有%s筆資料"%len(list(collection.find({}))))
                #將去重的資料再儲存於外部json檔
                
            #如果資料筆數不變，則停止繼續儲存的行為
            else:
                print("已存最大筆數")
                break
    except Exception as e:
        print(e)
        time.sleep(3)
        continue
    time.sleep(900)

In [4]:
#將資料讀出存成excel

import pandas as pd
import re

collection = db.wowprimeipeen
data=list(collection.find({}))

print("讀取完畢")

#加入一些資料
for i in data:
    try:
        i['othername']=re.findall("\((\w*[店|館|門市|百貨|街|夜市|商場|時代|廣場|中心|三越|大學|中|區|高鐵])\)",i['name'])[0]
    except:
        i['othername']=0
    if i['othername']!=0:
        i['name']=i['name'].split("(")[0]
    try:
        i["bigadd"]=re.findall("(..[市|縣])(\w\w?\w?[區|市|鎮|鄉])",
                               i["address"])[0][0]
#         i["bigadd"]=re.findall("(..[市|縣])(.+[區|市|鎮|鄉]).+[路|街|地段|巷][0-9]+[-|之]?[0-9]*?[號|巷|弄]?[0-9]*[號|巷|弄|樓]",
#                                i["address"])[0][0]
    except:
        i['bigadd']=0
    try:
        i["smalladd"]=re.findall("(..[市|縣])(\w\w?\w?[區|市|鎮|鄉])",
                           i["address"])[0][1]
    except:
        i["smalladd"]=0
#     if i["address"]==0 and i["status"]!="結束營業":
#         i["status"]="無地址可能已搬遷"
print("資料編輯完畢")
        
# Create a Pandas dataframe from some data.
df = pd.DataFrame(data)
df

df=df.reindex(columns=['id','name','othername','tele','address','bigadd',"smalladd",'lat','lng','bigstyle','smallstyle','averagecost',
                    'ipeenscore','status','Ncomment','reviewdate'])
# # Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('../data/ipeenpandas.xlsx', engine='xlsxwriter')

# # Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# # Close the Pandas Excel writer and output the Excel file.
writer.save()
print("儲存完畢")

讀取完畢
資料編輯完畢
儲存完畢


In [3]:
#存進mongo一次用
collection = db.wowprimeipeen
f = lambda x,y:x if y in x else x + [y]
with open("../data/diendata.json","r") as d:
    diendata=json.load(d)
    print("讀取完畢")
    #做去重動作
    diendata = reduce(f, [[], ] + diendata)
    print("去重完畢")
    with open("../data/mongodiendata.json","w") as d:
            json.dump(diendata,d)
    print("另存完畢")
    #比數變多，則做儲存動作
    if len(diendata)>len(list(collection.find({}))):
        collection.drop()
        collection.insert_many(diendata)
        print("存資料庫完畢")
        print("目前有%s筆資料"%len(list(collection.find({}))))

讀取完畢
去重完畢
另存完畢
存資料庫完畢
目前有166483筆資料


# 用mongodb的資料，修復原始json檔

In [ ]:
x=list(collection.find({}))
[i.pop("_id") for i in x]

with open("../data/diendata.json","w") as d:
    json.dump(x,d)

In [ ]:
# # 用mongodb的資料，修復原始json檔 修復地址
# from bs4 import BeautifulSoup as bs
# collection = db.wowprimeipeen
# x=list(collection.find({}))
# [i.pop("_id") for i in x]
# for i in x:
#     if i['address']==0 and i['lng']!=0:
#         try:
#             with open("./data/html2/"+str(i['id'])+".html","r",encoding='utf8') as f:
#                 dienhtml=f.read()
#                 soup=bs(dienhtml,"lxml")
#                 info=soup.select_one("#shop-header > div.info")
#                 i["address"]=info.find('a', attrs={'data-label': '上方地址'}).text.replace("\n"," ").strip()
# #                 print(i["address"])
#         except Exception as e:
#             print(e)

# # 用mongodb的資料，修復原始json檔 修復狀態
# from bs4 import BeautifulSoup as bs
# collection = db.wowprimeipeen
# x=list(collection.find({}))
# [i.pop("_id") for i in x]
# n=0
# for i in x:
#     if i['address']==0 and i['status']=="正常營業":
#         try:
#             with open("./data/html2/"+str(i['id'])+".html","r",encoding='utf8') as f:
#                 n+=1
#                 if n%50==0:
#                     print(n)
#                 dienhtml=f.read()
#                 soup=bs(dienhtml,"lxml")
#                 info=soup.select_one("#shop-header > div.info")
#                 if "已搬遷" in info.select_one("h1").text:
#                     i["status"]="已搬遷"
                
#         except Exception as e:
#             print(e)
            
            
# with open("./data/diendata.json","w") as d:
#     json.dump(x,d)


In [ ]:
collection.drop()
collection.insert_many(x)
[i.pop("_id") for i in x]
with open("../data/mongodiendata.json","w") as d:
    json.dump(x,d)
with open("../data/diendata.json","w") as d:
    json.dump(x,d)

In [ ]:
x=list(collection.find({}))
len(x)

In [ ]:
with open("../data/diendata.json","r") as d:
    diendata=json.load(d)
len(diendata)

# 第二版存取(含CommentURL)

In [ ]:
import pymongo
from pymongo import MongoClient
import datetime
import json
import time
from functools import reduce

#連Mongodb
client = MongoClient('172.20.26.39', 27017)
#連DB名
db=client['rawData']
#秀DB內的collection list
db.collection_names()


In [ ]:
#選擇我們想要的collection(亦可直接新增)
collection = db.wowprimeipeenAddCommentURL

In [ ]:
#list(dict)的去重公式
f = lambda x,y:x if y in x else x + [y]

loadPath='../data/diendataAddCommentUrl.json'
#不斷重複動作
while True:
    #讀取資料
    try:
        with open(loadPath,"r") as d:
            diendata=json.load(d)
            print("讀取完畢")
            #做去重動作
            diendata = reduce(f, [[], ] + diendata)
            print("去重完畢")
            with open("../data/mongodiendata2.json","w") as d:
                    json.dump(diendata,d)
            print("另存完畢")
            #比數變多，則做儲存動作
            if len(diendata)>len(list(collection.find({}))):
                collection.drop()
                collection.insert_many(diendata)
                print("存資料庫完畢")
                print("目前有%s筆資料"%len(list(collection.find({}))))
                #將去重的資料再儲存於外部json檔
                
            #如果資料筆數不變，則停止繼續儲存的行為
            else:
                print("已存最大筆數")
                break
    except Exception as e:
        print(e)
        time.sleep(3)
        continue
    time.sleep(900)

In [ ]:
x=list(collection.find({}))
[i.pop("_id") for i in x]
x